<a href="https://colab.research.google.com/github/spe301/AI-generated-AI/blob/main/AI_generated_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Potosnail==0.0.6

In [2]:
import pandas as pd
import numpy as np
from google.colab import files
import zipfile
import io
from potosnail import MachineLearning, DeepLearning, DataHelper, Evaluater, Algorithms, Wrappers

ml = MachineLearning()
dl = DeepLearning()
dh = DataHelper()
ev = Evaluater()
al = Algorithms()
wr = Wrappers()

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
df = pd.read_csv('https://raw.githubusercontent.com/spe301/Wikipedia-Capstone/main/Data/HumanText.csv').drop(['Unnamed: 0', 'Unnamed: 0.1'], axis='columns')
df2 = pd.read_csv('https://raw.githubusercontent.com/spe301/Wikipedia-Capstone/main/Data/Generated%20(1).csv').drop(['Unnamed: 0'], axis='columns')
blog_df = pd.concat([df.sample(n=180), df2])

df = pd.read_csv('https://raw.githubusercontent.com/spe301/dsc-phase-4-project/main/data/judge-1377884607_tweet_product_company.csv', engine='python',encoding='latin-1')
df = df.drop(6)
df = df.loc[df['is_there_an_emotion_directed_at_a_brand_or_product'] != "I can't tell"]
ix = list(df.index)
clean = []
for i in range(len(ix)):
  clean.append(df['tweet_text'][ix[i]].replace('.@', '').replace('.', '').replace('!', '').replace('@', '').replace('?', '').replace(':', '')
  .replace(',', '').replace(';', '').lower())

df['tweet'] = clean
tweet_df = df.drop(['tweet_text', 'emotion_in_tweet_is_directed_at'], axis='columns')

In [23]:
blog_df.to_csv('drive/MyDrive/Wikipedia.csv', index=False)

In [ ]:
def Preprocess(df, data_str, target_str):
  '''fully preprocesses a dataframe of Text Documents for classification'''
  avg = 0
  for i in range(len(df[data_str])):
    avg += len(list(df[data_str])[i].split(' '))/len(df)
  pad = int(avg)
  text, labels = dl.ModelReadyText1(df[data_str], df[target_str], pad)
  return text, labels

blogs, blog_labels = Preprocess(blog_df, 'Text', 'AI')
tweets, tweet_labels = Preprocess(tweet_df, 'tweet', 'is_there_an_emotion_directed_at_a_brand_or_product')

In [ ]:
def GetVocab(df, data_str):
  words = []
  for i in range(len(df)):
    word_lst = list(df[data_str])[i].replace('\n', ' ').split(' ')
    for word in word_lst:
      words.append(word.replace('.', '').replace(',', '').replace(' ', '').replace('"', '').replace(':', '').replace(';', '').replace('!', ''))
  return len(np.unique(words))

In [ ]:
blogs_vocab = GetVocab(blog_df, 'Text')
tweets_vocab = GetVocab(tweet_df, 'tweet')

In [ ]:
#uploaded = files.upload()
#data = zipfile.ZipFile(io.BytesIO(uploaded['IMGDS1.zip']), 'r')
#data.extractall()

Saving IMGDS1.zip to IMGDS1.zip


In [ ]:
train_dir = r'dataset/training_set'
test_dir = r'dataset/test_set'
train_dogs = r'dataset/training_set/dogs'
train_cats = r'dataset/training_set/cats'
test_dogs = r'dataset/test_set/dogs'
test_cats = r'dataset/test_set/cats'
directories = [train_dir, test_dir, train_dogs, train_cats, test_dogs, test_cats]
classes = ['cats', 'dogs']

tri1, tei1, tray1, tey1 = dl.ModelReadyPixles(directories, classes)

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [10]:
from sklearn.datasets import load_breast_cancer, load_boston

cancer_data = load_breast_cancer()['data']
diagnosis = dl.MulticlassOutput(load_breast_cancer()['target'])
housing_data = load_boston()['data']
prices = load_boston()['target']

# Parameter Grid Templates

rnn_grid = {'output_dim': [labels.shape[1]], 'embedding': [vocab], 'nodes': [int], 'activation': [str], 
              'regularizer': [None or str], 'stacking': [bool], 'dropout': [bool], 'optimizer': [str], 
              'method': [str], 'bidirectional': [bool]} 

cnn_grid = {'output_dim': [lables.shape[1]], 'base_filters':[int], 'kernel_size': [int], 'activation': [str], 
            'nblocks': [int], 'pool':[int], 'dropout': [bool], 'closer': [bool], 'optimizer': [str], 
            'metrics': [str]}

tc_grid = {'output_dim': [lables.shape[1]], 'nodes': [int], 'activation': [str], 'regularizer': [None or str], 'stacking': [bool], 
               'dropout': [bool], 'nlayers': [int], 'closer': [bool], 'loss': [str], 'optimizer': [str]}

tr_grid = {'nodes': [int], 'activation': [str], 'regularizer': [None or str], 'stacking': [bool], 'dropout': [bool], 'nlayers': [int], 'closer': [bool], 
                'loss': [str], 'optimizer': [str], 'y_col': [str]}

# Image dataset directory graveyard
copy and paste these into a code cell when working with the dataset

## Dogs and Cats
train_dir = r'dataset/training_set'

test_dir = r'dataset/test_set'

train_dogs = r'dataset/training_set/dogs'

train_cats = r'dataset/training_set/cats'

test_dogs = r'dataset/test_set/dogs'

test_cats = r'dataset/test_set/cats'

directories = [train_dir, test_dir, train_dogs, train_cats, test_dogs, test_cats]

classes = ['cats', 'dogs']